In [1]:
import os
from llama_index import download_loader
from langchain.llms import OpenAI
import openai
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from pathlib import Path
import pinecone
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from tqdm.auto import tqdm
import tiktoken
from pathlib import Path
from gpt_index import download_loader
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [ ]:
url = "https://langchain-internal-project-" + whoami_data.projectname + ".svc.us-west4-gcp.pinecone.io"

In [5]:
limit = 3750
embed_model = "text-embedding-ada-002"

def retrieve(query):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )

    # retrieve from Pinecone
    xq = res['data'][0]['embedding']

    # get relevant contexts
    res = index.query(xq, top_k=3, include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in res['matches']
    ]

    print(contexts)

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    
    prompt = (prompt_start + "\n\n---\n\n".join(contexts) + prompt_end)

    return prompt

In [6]:
def complete(prompt):
    # query text-davinci-003
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=0,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

In [7]:
#first we retrieve relevant items from Pinecone
query = "Can you summarize february 9 from this book"
query_with_contexts = retrieve(query)
query_with_contexts

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.

In [ ]:
#then we complete the context-infused query
complete(query_with_contexts)